### CountVectorizer

In [1]:
import re
import pandas as pd
import numpy as np
import os
from datetime import datetime
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
import gensim.corpora as corpora
from sentence_transformers import SentenceTransformer
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Load data
lucy = pd.read_csv('/workspace/data/실시간검색어20230110.csv')
lucy['내용'] = lucy['내용'].replace(np.nan, '')
lucy['내용'] = lucy['내용'].replace("\n", '')
# Filter

lucy.내용 = lucy.apply(lambda row: " ".join(re.sub("[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…《\》]", " ", row.내용).split()), 1)
lucy.내용 = lucy.apply(lambda row: " ".join(re.sub("([ㄱ-ㅎㅏ-ㅣ]+)", " ", row.내용).split()), 1)
lucy.내용 = lucy.apply(lambda row: " ".join(re.sub("([♡❤✌❣♥ᆢ✊❤️✨⤵️☺️;”“]+)", " ", row.내용).split()), 1)
lucy.내용 = lucy.apply(lambda row: " ".join(re.sub("_x000D_", "", row.내용).split()), 1)
timestamps = lucy.수집일.to_list()
contents_data = lucy.내용.to_list()

In [6]:
from typing import List

def _preprocess_text(self, documents: np.ndarray) -> List[str]:

    cleaned_documents = [doc.replace("\n", " ") for doc in documents]
    cleaned_documents = [doc.replace("\t", " ") for doc in cleaned_documents]
    if self.language == "english":
        cleaned_documents = [re.sub(r'[^A-Za-z0-9 ]+', '', doc) for doc in cleaned_documents]
    cleaned_documents = [doc if doc != "" else "emptydoc" for doc in cleaned_documents]
    return cleaned_documents

In [ ]:
documents = pd.DataFrame({"Document": contents_data,
                          "ID": range(len(contents_data)),
                          "Topic": topics})

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=10)

In [11]:
vcorpus=vectorizer.transform(contents_data)

In [14]:
vcorpus.shape

(4097, 163185)

In [12]:
topic_model = SentenceTransformer('jhgan/ko-sbert-sts')
corpus = topic_model.encode(vcorpus)

TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

In [ ]:
vcorpus=vectorizer.fit(corpus)

In [ ]:
df = train_data

topics=['인공위성', '지진', '미국증시', '더글로리', '송중기', '너클', '성남의혹']
df['topics'] = [topics.index(line) for line in df['검색어']]

In [ ]:
import umap
import plotly.express as px
import hdbscan

umap_embeddings = umap.UMAP(n_neighbors=140, 
                            n_components=2, 
                            metric='cosine').fit_transform(corpus)

# cluster = hdbscan.HDBSCAN(min_cluster_size=30,
#                           metric='euclidean',                      
#                           cluster_selection_method='eom').fit(umap_embeddings)

# print(cluster.labels_.max())

In [ ]:
import plotly.express as px

result = pd.DataFrame(umap_embeddings, columns=['x', 'y'])
result['labels'] = cluster.labels_
result['subject'] = lucy['검색어']
fig = px.scatter(result, x='x', y='y', color='labels', hover_data=['subject'], width=750, height=750)
fig.show()